# Word frequencies


Now that we have the abstracts in two nice neat .csv files, we need to download/import the packages needed, import the .csv files, and then can get on with the first part of the analysis. 

## Get ready 

As always, we start with a couple of code cells that load up and nickname some useful packages, then check file locations, then import files and check them. 


In [1]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk


In [2]:
%%capture

import os                         # os is a module for navigating your machine (e.g., file directories).
import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions

nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics
import datetime
date = datetime.date.today()

import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections
import matplotlib.pyplot as plt
import string 

import math 

English_punctuation = "-!\"#$%&()'*-–+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)

In [4]:
print(os.listdir("..\\output")  )                                # check 'results' folder is not empty/has correct stuff

['.ipynb_checkpoints', 'all_abstracts_no_null_texts.csv']


## Import

Having checked the contents of the output folder and seen the files we expected to see, we can now import and check them. 

In [5]:
all_texts = pd.read_csv('..\\output\\all_abstracts_no_null_texts.csv')            # one for all of the texts and then


In [6]:
print (len(all_texts))                        # it is always useful to double check that the length matches your expectations


38228


## Get some basic stats about how texts are spread out over time

We know that all of the rows in the files have at least two columns with contents - 'Year' and 'Text'. This means that it is probably a useful thing to get a little schematic and/or table that counts row according to year. Let's do that now!  

In [7]:
all_counts_by_year = all_texts['Year'].value_counts()         # this creates a little table with two columns - year and count
print(all_counts_by_year)                                     # however, when we print it we can see it has no headers,
                                                              # is not in order, has the years appearing as  floats, etc. 

2021.0    2524
2013.0    2373
2001.0    2336
2014.0    2240
2004.0    2205
2016.0    2040
2011.0    1967
2015.0    1951
2008.0    1896
2012.0    1871
2020.0    1853
2010.0    1716
2009.0    1704
2018.0    1617
2017.0    1601
2019.0    1589
2007.0    1541
2005.0    1520
2006.0    1422
2002.0    1266
2003.0     996
Name: Year, dtype: int64


In [8]:
all_counts_by_year = pd.DataFrame(all_counts_by_year)                            # Convert the imported table to a data frame,
all_counts_by_year = all_counts_by_year.rename(columns={"Year": "All"})          # rename the columns,
all_counts_by_year = all_counts_by_year.rename_axis('Year').reset_index()        # set the axis to 'Year' and reset the index,
all_counts_by_year = all_counts_by_year.sort_values(by=['Year']).astype('Int64') # retype 'Year' column and sort by year.
print(all_counts_by_year)                                                        # Let's just check it worked. 

    Year   All
2   2001  2336
19  2002  1266
20  2003   996
4   2004  2205
17  2005  1520
18  2006  1422
16  2007  1541
8   2008  1896
12  2009  1704
11  2010  1716
6   2011  1967
9   2012  1871
1   2013  2373
3   2014  2240
7   2015  1951
5   2016  2040
14  2017  1601
13  2018  1617
15  2019  1589
10  2020  1853
0   2021  2524


## Count word frequencies - 'bag of words'

Now that we have some basic descriptive stats about how many abstracts were imported properly with text in the 'Text' column, we can get on to the actual natural language processing steps. The most basic NLP option is to count the most frequent words found in the two sets of abstracts - meaning we need to find the most frequent words found in ALL of the abstracts and then compare that to the most frequnet words found in only those abstracts that contain a keyword of interest. 

To this end, we use the 'bag of words' method which whacks all of the words from all of the texts together, turns them into 'tokens' then processes to make them as unified as possible by removing uppercase letters, punctuation, digits, empty strings, stop words (e.g. 'the', 'and', 'for', etc. ) and word forms (e.g. pluralisations, verb endings, etc. ). 

Let's demo this with a simple example. If the text we want to 'bag of words' is "The cat named Cat was one of 5 cats." it would become a list of stemmed word-tokens like 
'''[[cat]
[name]
[cat]
[be]
[cat]]''' 
and the most common word would obviously be '''[cat]'''. 

Applying the 'bag of words' method to our texts is not so trivial, but should also be more enlightening. We would expect that the most common words from all of the texts would be similar to, but not identical to, the most common words from only the abstracts that contain a keyword of interest.

This bag of words approach ignores years, session codes, authors and everything else. Subsetting the texts by those things might be useful later. 

In [9]:
def bag_of_words_analysis(input, how_many):     # define a 'bag of words' function with 2 arguments, an input and a quantity 
    holding_string = ""                                                        # that creates a temporary variable
    for text in input['Text']:                                                 # looks at the 'Text' column for the input
        holding_string += text                                                 # fills up the temp variable with the text
    holding_string = word_tokenize(holding_string)                             # word tokenises that text
    holding_string = [word.lower() for word in holding_string]                 # remove uppercase letters
    holding_string = [w.translate(table_punctuation) for w in holding_string]  # removes punctuation
    holding_string = (list(filter(lambda x: x, holding_string)))               # removes andy empty strings
    holding_string = [token for token in holding_string if not token.isdigit()]  # removes digits
    holding_string = [token for token in holding_string if token not in stop_words]  # removes stopwords
    holding_string = [porter.stem(token) for token in holding_string]                # stems the word-tokens
    list_for_count = []                                                              # and creates an empty list
    for token in holding_string:                                         # then iterates over the tokens
        list_for_count.append(token)                                     # appending them to the list
    counts = Counter(list_for_count)                                     # applies the Counter function imported earlier 
    return counts.most_common(how_many)                                  # and returns the tokens with highest counts 
                                                                         # up to the quantity specified as an argument

In [10]:
most_frequent_all = bag_of_words_analysis(all_texts, 20)   # apply bag of words function to all texts, and save the output table
                                                           # this will take a while. 

### Steps to take

* revise the bag_of_words_analysis function to group by year? ALternatively, create one bag of words for each year. 
* write a new function that scans the year-bags-of-words for all words on an input list
* save the output of that function to a list (or .csv?) 
* use the outputs to create graphs that track the popularity of all the words on that list over time



In [ ]:
print(counts['autist'])
print(counts['asd'])
print(counts['asperg'])
print(counts['autism'])